In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
train2 = pd.read_csv('train_final.csv')
train = pd.DataFrame(train2.loc[245:])
test = pd.read_csv('test_final.csv')

In [2]:
dict, dict1 = [], []
for element in train2['Tectonic regime']:
    strs = element.split("/")
    for str in strs:
        if dict.count(str) == 0:
            dict.append(str)
for typ in dict:
    for index in train.index:
        train.at[index, typ] = 1 if train.at[index, 'Tectonic regime'].split('/').count(typ) != 0 else 0
        
for element in train2['Structural setting']:
    strs = element.split("/")
    for str in strs:
        if dict1.count(str) == 0:
            dict1.append(str)
for typ in dict1:
    for index in train.index:
        train.at[index, typ] = 1 if train.at[index, 'Structural setting'].split('/').count(typ) != 0 else 0

for name in ['Onshore/Offshore', 'Hydrocarbon type', 'Reservoir status', 'Period', 'Lithology']:
    label_encoder = preprocessing.LabelEncoder() 
    train[name] = label_encoder.fit_transform(train[name]);

In [3]:
for typ in dict:
    for index in test.index:
        test.at[index, typ] = 1 if test.at[index, 'Tectonic regime'].split('/').count(typ) != 0 else 0
for typ in dict1:
    for index in test.index:
        test.at[index, typ] = 1 if test.at[index, 'Structural setting'].split('/').count(typ) != 0 else 0

for name in ['Hydrocarbon type', 'Reservoir status', 'Period', 'Lithology']:
    label_encoder = preprocessing.LabelEncoder() 
    test[name] = label_encoder.fit_transform(test[name]);

In [4]:
X1 = train.drop(['Onshore/Offshore', 'Structural setting', 'Tectonic regime'], axis=1)
test = test.drop(['Structural setting', 'Tectonic regime'], axis = 1)
y1 = train['Onshore/Offshore']
"""
scaler = MinMaxScaler([0,1])
X_st = scaler.fit_transform(X)
"""
scaler = StandardScaler()
X_st = scaler.fit_transform(X1)
oversample = SMOTE(k_neighbors=4)
X, y = oversample.fit_resample(X_st, y1)
scaler2 = StandardScaler()
test = scaler2.fit_transform(test)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
answers_pred = model.predict(X_test)
accuracy_score(y_test,answers_pred)

0.8

In [6]:
predictions = model.predict(test)
dic = ['OFFSHORE', 'ONSHORE', 'ONSHORE-OFFSHORE']
a, b, y = [], [], 0
for predict in predictions:
    a.append(y)
    b.append(dic[predict])
    y += 1
final_prediction = pd.DataFrame({'index':a,'Onshore/Offshore':b})
final_prediction.shape
final_prediction.to_csv('final_prediction.csv', index=False)